* TabPFNと（デフォルトの）XGBoostのアンサンブルを使用する。
* このコンペティションで使用されるバランスの取れた対数損失に一致するように確率を重み付けし直す
* 欠損値は中央値で補完
* 訓練データの時間列を使用, テストデータでは訓練データの時間列+1とする
* greeks.Alphaで提供される4つのクラスすべてを使用し、後者の3つのクラスについては確率を集約する。

* <https://www.kaggle.com/code/vadimkamaev/postprocessin-ensemble>
* <https://www.kaggle.com/code/aikhmelnytskyy/public-krni-pdi-with-two-additional-models>
* <https://www.kaggle.com/code/maverickss26/lb-0-06-icr>
* <https://www.kaggle.com/code/siddhvr/xgb-tabpfn-improved>
* <https://www.kaggle.com/code/renatoreggiani/icr-bestpublicscore-add-xgb-lgbm>

## TabPFNのインストール
* 事前にダウンロードするファイル: <https://www.kaggle.com/datasets/carlmcbrideellis/tabpfn-019-whl>

In [1]:
# !pip install -q /kaggle/input/tabpfn-019-whl/tabpfn-0.1.9-py3-none-any.whl
# !mkdir /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
# !cp /kaggle/input/tabpfn-019-whl/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
# model
from sklearn.base import BaseEstimator
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from tabpfn import TabPFNClassifier
# over/under sampling
from imblearn.over_sampling import SMOTE # SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# Imputation
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif# Feature Selection
import category_encoders as encoders
from sklearn.preprocessing import LabelEncoder, RobustScaler
# others
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from datetime import date, datetime
import optuna
import warnings
warnings.simplefilter('ignore')

# 環境を指定
env = 'local'
# env = 'kaggle'

In [13]:
# ディレクトリの指定
if env == 'local':
    BASE_DIR = '../../data'
elif env == 'kaggle':
    BASE_DIR = '/kaggle/input/icr-identify-age-related-conditions/'
else:
    raise ValueError("Invalid environment. Set env as 'local' or 'kaggle'.")

# データの読み込み
train_df = pd.read_csv(f'{BASE_DIR}/train.csv')
# train_df = pd.read_csv(f'{BASE_DIR}/train_integerized.csv')
greeks_df = pd.read_csv(f'{BASE_DIR}/greeks.csv')
test_df = pd.read_csv(f'{BASE_DIR}/test.csv')
submission_df = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

# greeksと結合
train_df = pd.merge(train_df, greeks_df[['Id', 'Alpha', 'Epsilon']], on='Id', how='left')

In [14]:
# 欠損値以外の日付をグレゴリオ暦の序数形式（1年1月1日を1とし、1日ずつ増やしていく）に変換
train_df.Epsilon[train_df.Epsilon != 'Unknown'] = train_df.Epsilon[train_df.Epsilon != 'Unknown']\
                                        .map(lambda x: datetime.strptime(x, '%m/%d/%Y').toordinal())
# 欠損値をnp.nanに変換
train_df.Epsilon[train_df.Epsilon == 'Unknown'] = np.nan

# 訓練データを説明変数と目的変数に分割
X_train = train_df.drop(['Id', 'EJ', 'Alpha', 'Class'], axis=1)
y_train = train_df[['Class', 'Alpha']]

# テストデータから数値データ以外を削除
X_test = test_df.drop(['Id', 'EJ'], axis=1)

# テストデータは訓練データの最大値+1とする
X_test['Epsilon'] = train_df.Epsilon.max()+1

In [15]:
class WeightedEns(BaseEstimator):
    def __init__(self):
        self.models = [XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.2,subsample=0.9,colsample_bytree=0.85),
                       TabPFNClassifier(N_ensemble_configurations=256,device='cuda:0')]
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        # self.imputer = KNNImputer(n_neighbors=50)
    
    def fit(self, X, y):
        classes, y = np.unique(y, return_inverse=True)
        self.classes_ = classes
        X = self.imputer.fit_transform(X)
        for model in self.models:
            model.fit(X,y)
    
    def predict_proba(self, X):
        X = self.imputer.transform(X)
        ps = np.stack([model.predict_proba(X) for model in self.models])
        p = np.mean(ps,axis=0)
        class_0_est_instances = p[:,0].sum()
        others_est_instances = p[:,1:].sum()
        # we reweight the probs, since the loss is also balanced like this
        # our models out of the box optimize CE
        # with these changes they optimize balanced CE
        new_p = p * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(p.shape[1])]])
        new_p = new_p / np.sum(new_p,axis=1,keepdims=1)
        return np.concatenate((new_p[:,:1],np.sum(new_p[:,1:],1,keepdims=True)), 1)

In [16]:
# 評価基準
def balanced_log_loss(y_true, y_pred):
    N = len(y_true)

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.maximum(np.minimum(y_pred, 1 - 1e-15), 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1-y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

In [19]:
# 各分割ごとのバリデーションスコアを格納
scores = 0
# モデルを保存
models = []
# クロスバリデーションの分割で考慮する特徴量
labels = greeks_df[['Beta', 'Gamma', 'Delta']]
# クロスバリデーションの分割数を指定します
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=779292)

for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, labels)):
    # 進行状況
    print('fold: {}'.format(fold+1))
    # 訓練データを分割
    X_train_fold = X_train.iloc[train_index]
    y_train_fold = y_train['Alpha'].iloc[train_index]
    X_valid_fold = X_train.iloc[valid_index]
    y_valid_fold = y_train['Class'].iloc[valid_index]
    
    # モデルを訓練、予測を出力
    model = WeightedEns()
    model.fit(X_train_fold,y_train_fold)
    valid_preds = model.predict_proba(X_valid_fold)[:, 1]

    # 後処理
    boost = 2.7 # TRY DIFFERENT FACTORS WITH YOUR MODEL
    odds = boost * valid_preds / (1-valid_preds)
    valid_preds = odds / (1+odds)

    # 評価
    val_score = balanced_log_loss(y_valid_fold, valid_preds)
    # スコアを保存
    scores += val_score
    # モデルを保存
    models.append(model)
    
# クロスバリデーションの平均値を計算
cv_score = scores /  12
print(f'our out of folds CV score is {cv_score}')

fold: 1
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
fold: 2
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
fold: 3
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
fold: 4
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
fold: 5
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
our out of folds CV score is 0.09393895471301907


In [8]:
# 提出用に値を変換
if env == 'kaggle':
    # 予測
    # 各分割ごとのテストデータに対する予測値を格納
    preds = np.zeros(len(X_test))
    for i in range(len(models)):
        # pred = models[i].predict(xgb.DMatrix(test_df.drop(['Id', 'EJ'], axis=1)), iteration_range=(0, models[i].best_iteration))
        pred = models[i].predict(X_test)
        preds += pred
    test_pred = preds / 12

    # 提出
    submission = pd.DataFrame(columns = submission_df.columns)
    submission['Id'] = test_df['Id']
    submission['class_0'] = 1 - test_pred
    submission['class_1'] = test_pred
    submission.to_csv('submission.csv',index=False)